In [1]:
import os, gc
import torch
from torch.utils.data import DataLoader, Dataset
from nlp_datasets import get_dataset, get_dataloader
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset, load_metric
from transformers import DataCollatorForLanguageModeling
from tensorboardX import SummaryWriter

# Initialize TensorBoard
writer = SummaryWriter()


# Load the dataset
train_dataset = get_dataset() 
valid_dataset = get_dataset(split = "valid")


# Add padding token
# tokenizer.pad_token = tokenizer.eos_token

text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(text_decoder_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Data collator to dynamically pad the inputs to the maximum length of the batch
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[손해배상(기)]〈일제 강제동원 피해자들이 일본 기업을 상대로 불법행위로 인한 위자료 지급을 구하는 사건〉 [공2024상,204]\n판시사항:[1] 객관적으로 채권자가 권리를 행사할 수 없는 장애사유가 있었던 경우, 채무자가 소멸시효 완성을 주장하는 것이 신의성실의 원칙에 반하는 권리남용에 해당하는지 여부(적극)\n[2] 채권자에게 권리의 행사를 기대할 수 없는 객관적인 사실상의 장애사유가 있었으나 대법원이 채권자의 권리행사가 가능하다는 법률적 판단을 내린 경우, 그 시점 이후에는 장애사유가 해소되었다고 볼 수 있는지 여부(원칙적 적극)\n[3] 일제강점기에 강제동원되어 기간 군수사업체인 구 미쓰비시중공업 주식회사에서 강제노동에 종사한 갑 등이 위 회사가 해산된 후 새로이 설립된 미쓰비시중공업 주식회사를 상대로 위자료 지급을 구한 사안에서, 강제동원 피해자의 일본 기업에 대한 위자료청구권은 ‘대한민국과 일본국 간의 재산 및 청구권에 관한 문제의 해결과 경제협력에 관한 협정’의 적용 대상에 포함되지 않는다는 법적 견해를 최종적으로 명확하게 밝힌 대법원 2018. 10. 30. 선고 2013다61381 전원합의체 판결이 선고될 때까지는 강제동원 피해자 또는 그 상속인들에게는 미쓰비시중공업 주식회사를 상대로 객관적으로 권리를 사실상 행사할 수 없는 장애사유가 있었다고 봄이 타당하다고 한 사례\n\n        assistant : \n    ', 'label': '\n그러므로 상고를 모두 기각하고 상고비용은 패소자가 부담하도록 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유(상고이유서 제출기간이 지난 후에 제출된 상고이유보충서의 기재는 상고이유를 보충하는 범위 내에서)를 판단한다

Map:   0%|          | 0/815 [00:00<?, ? examples/s]

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[재산세부과처분취소]〈오피스텔을 주택으로 취급하여 재산세를 부과할 수 있는지 문제된 사건〉 [공2024상,64]\n판시사항:오피스텔이 재산세 과세대상인 ‘주택’에 해당하는지 판단하는 기준\n\n        assistant : \n    ', 'label': '\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유를 판단한다.\n1. 관련 법리\n지방세법 제105조, 제104조 제2호, 제6조 제4호, 건축법 제2조 제1항 제2호, 제2항 및 그 위임에 따른 건축법 시행령 제3조의5 [별표 1] 제14호는 재산세 과세대상인 ‘건축물’ 중 일반업무시설의 하나로 ‘오피스텔(업무를 주로 하며, 분양하거나 임대하는 구획 중 일부 구획에서 숙식을 할 수 있도록 한 건축물로서 국토교통부장관이 고시하는 기준에 적합한 것을 말한다)’을 명시하였고, 지방세법 제105조, 제104조 제3호, 구 주택법(2021. 12. 21. 법률 제18631호로 개정되기 전의 것, 이하 같다) 제2조 제1호는 재산세 과세대상인 ‘주택’을 ‘세대의 구성원이 장기간 독립된 주거생활을 할 수 있는 구조로 된 건축물의 전부 또는 일부 및 그 부속토지’로 정하면서, 재산세 과세대상인 토지와 건축물의 범위에서 주택은 제외한다고 규정하였다.\n구 지방세법 시행령(2021. 12. 31. 대통령령 제32293호로 개정되기 전의 것) 제119조는 ‘재산세의 과세대상 물건이 공부상 등재 현황과 사실상의 현황이 다른 경우에는 사실상 현황에 따라 재산세를 부과한다.’고 규정하였으므로, 재산세 과세대상인 ‘주택’에 해당하는지 여부는 특별한 사정이 없는 한 공부상의 용도에 관계없

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
for name, module in base_model.named_modules():
    print(name)

NameError: name 'base_model' is not defined

In [4]:
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs.loss
        
        return (loss, outputs) if return_outputs else loss

In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    # quantization_config=bnb_config
    )

####---------------LoRA-------------###
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    # modules_to_save=["q_proj", "k_proj", "v_proj"],
)
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=False,
    dataloader_num_workers=4,
    report_to="none",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-5
)

# Check if CUDA is available and use it
if torch.cuda.is_available():
    model.cuda()


trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

# Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
#     eval_dataset=valid_dataset,
# )

# Train the model
trainer.train()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 4,718,592 || all params: 8,172,867,584 || trainable%: 0.0577


/home/eahc00/venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parall

Epoch,Training Loss,Validation Loss
1,0.000000,No log
2,0.000000,No log
3,0.000000,No log
4,0.000000,No log
5,0.000000,No log
6,0.000000,No log
7,0.000000,No log
8,0.000000,No log
9,0.000000,No log
10,0.000000,No log


/home/eahc00/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, 

TrainOutput(global_step=16300, training_loss=0.0015118299378939202, metrics={'train_runtime': 5992.7766, 'train_samples_per_second': 2.72, 'train_steps_per_second': 2.72, 'total_flos': 3.967517685959885e+17, 'train_loss': 0.0015118299378939202, 'epoch': 20.0})

In [6]:
# Save the model and tokenizer
model.save_pretrained("./results/latest_checkpoint")
tokenizer.save_pretrained("./results/latest_checkpoint")

# Evaluate the model on the test dataset
rouge = load_metric("rouge")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits, labels = torch.tensor(logits), torch.tensor(labels)

    predictions = torch.argmax(logits, dim=-1)
    # Remove all special tokens including padding and None values from labels
    labels = [[token for token in label if token != -100] for label in labels.tolist()]

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects newline after each sentence
    decoded_preds = ["\n".join(pred.strip() for pred in preds.split(". ")) for preds in decoded_preds]
    decoded_labels = ["\n".join(label.strip() for label in labels.split(". ")) for labels in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {key: value.mid.fmeasure * 100 for key, value in result.items()}

# Trainer for evaluation
test_trainer = Trainer(
    model=model,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Evaluate the model on the test dataset
test_results = test_trainer.predict(test_dataset)

# Print the results
print("Test ROUGE Scores:", test_results.metrics)

# Close the TensorBoard writer
writer.close()

# Release GPU memory
del model
del trainer
del test_trainer
torch.cuda.empty_cache()
gc.collect()

/home/eahc00/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/tmp/ipykernel_1100430/2017558957.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/home/eahc00/venv/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be m

ImportError: To be able to use rouge, you need to install the following dependencies: nltk, rouge_score.
Please install them using 'pip install # noqa: F401 # Here to have a nice missing dependency error message early on rouge_score' for instance.